In [60]:
import pandas as pd
import re
import os

df = pd.read_csv('../csv_arrangement/test.csv')

test_text = df.판례내용[6] 
print(test_text)

crime = re.split(r'\n사 *건\b|\n피 *고 *인\b', maxsplit=2, string=test_text)[1]

print(crime)

인  천  지  방  법  원
판          결
사       건 2007고단5780  횡령
피  고  인 이○○
검       사
변  호  인 법무법인
판 결 선 고 2008. 4. 25.
  
주       문
피고인은 무죄.
  
이       유
1. 공소사실
피고인은 피해자 박○○에게 차용금 등으로 436,069,000원의 채무를 지고 있었다. 
피고인은 2006. 9. 13.경 인천 남구 󰏬󰏬󰏬에 있는 이○○ 변호사 사무실에서 위 채무
의 변제를 위해, 충남 서천군 서천화력발전소 진공펌프 설치 및 배관 공사로부터 발생
할 예정인 피고인의 임○○에 대한 공사대금 채권 중 100,000,000원의 공사대금 채권
을 양도하였다. 
그러함에도 불구하고, 장○○을 통하여 임○○으로부터 위 공사대금으로 2006. 10. 20.
경 29,700,000원, 같은 해 11. 3.경 500,000원, 같은 해 11. 6.경 46,200,000원, 같은 해 
- 1 -12. 11.경 7,000,000원, 2007. 1. 10.경 3,450,000원, 같은 해 1. 17.경 2,000,000원, 같
은 해 1. 23.경 53,550,000원, 같은 해 3. 30.경 3,000,000원 등 합계 145,400,000원을 
지급받아 보관하던 중 그 무렵 임의로 피고인의 공사비 등으로 모두 소비함으로써 
100,000,000원을 횡령하였다. 
2. 판단
이 사건 공소사실에 관하여 변호인은, 피고인이 채권양도 후 그 통지까지 마쳤으므로, 
피고인이 그 후 채무자로부터 양도된 채권을 변제명목으로 수령하였다 하더라도 위 돈
에 관하여 채권양수인을 위한 보관자라고 할 수 없으므로, 횡령죄는 성립하지 않는다
고 다툰다. 
살피건대, 검사가 제출한 채권양도계약서 사본, 채권양도통지서 사본의 각 기재에 의하
면, 피고인은 공소사실 기재와 같이 피고인의 임○○에 대한 공사대금 채권 중 1억원
을 박○○에게 양도하고, 같은 날 채무자인 임○○에게 위 채권양도사실을 통지한 사
실이 인정

In [70]:
print(crime)
test = re.compile(r'\b\d+[가-힣]{1,3}\d+\b')
print(test.search(crime))

 2007고단5780  횡령
<re.Match object; span=(1, 11), match='2007고단5780'>


In [58]:
print(crime)

 2020고단1142  상해


In [59]:
print(crime)

 2020고단1142  상해
